<a href="https://colab.research.google.com/github/Tharindu021/Context_Based_Spell_Checker/blob/main/SpellChecker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import NLTK is a natural language toolkit and the wordnet is a database it can use for the spell checker and also lingustic events.and from that we can get edit distance method also

In [2]:
import nltk
from nltk.corpus import wordnet
from nltk.metrics.distance import edit_distance
import pandas as pd
import gensim

i try to check with in the dataset in the drive.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


in this we can retrive the data from the dataset to use to the spell checking and then finally returns as the array

In [12]:
path = "/content/drive/MyDrive/Dataset/english Dictionary only words.csv"

df = pd.read_csv(path)
dictionary = set(df['word'].str.lower())
print(df['word'].tail())
#with open(path,'r') as file:
#    words = file.read().split()
#  return set(words)


<ipython-input-12-7a37bfac810e>:3: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


188131    Yupon
188132      Yux
188133     Yvel
188134     Ywar
188135     Ywis
Name: word, dtype: object


punkt is a tokenizer and we need to download that and the wordnet database also then we can preprocessing our project

In [5]:
nltk.download('punkt')
#nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

from below code we can get the input sentence and that sentence can tokenize using nltk.word_tokenize then we need to chech that words one by one then if it is not in the dataset then we can print there is no word like user input and show then to the what word are the mispelled and the recomendations using edit distance algorithm

In [6]:
def spell_checker(sentence, dictionary):
      simillar_word = []
      words = nltk.word_tokenize(sentence)
      for word in words:
          if word.lower() not in dictionary:
             simillar_word = recomend_simillar_words(word.lower())
             print("Recommended words for '{}' are: {}".format(word, simillar_word))

in this code we can get the simmilar words for misspelled words that are input have i choose only below 2 distance. and then recomend to the client only 10 words.

In [7]:
def recomend_simillar_words(words):
  simillar_words = []
  for dict_words in dictionary:
    distance = edit_distance( dict_words , words )
    if distance <= 2:
      simillar_words.append( (dict_words , distance) )
  simillar_words.sort(key=lambda x : x[1])
  return [ word[0] for word in simillar_words[:20]]

in that code part we can input the sentence then it strictly move to the spell_checker code part then it can check whether the word is in the dataset or not

In [16]:
user_input = ""
while user_input.lower() != "exit":
    user_input = input("Please enter the sentence you want or type 'exit' to quit: ")
    if user_input.lower() == "exit":
        print("Exiting...")
    else:
        spell_checker(user_input, dictionary)

Please enter the sentence you want, or type 'exit' to quit: exit
Exiting...
